# VacationPy
### Moriah Taylor
#### April 24, 2021

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import API key
from config import api_key

### Store Part I results into DataFrame

In [2]:
#specify path
path = "../WeatherPy/output_data/cities.csv"
#import csv file from part 1
city_df = pd.read_csv(path)

### Humidity Heatmap

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=api_key)

In [12]:
#store lat and lng into locations
locations = city_df[['Lat','Lng']].astype(float)
#convert humidity to float
humidity = city_df['Humidity'].astype(float)
#assign max humidity to variable
max_humidity = max(humidity)

#heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame Fitting My Idea Weather Criteria
My ideal weather is warm to hot, moderate humidity, low wind speed, and light cloudiness, so I subset to these conditions.

In [13]:
#drop null values (shouldn't be any)
reduced_df = city_df.dropna(axis=0, how='all')
#subset to my ideal weather conditions
reduced_df = reduced_df[(reduced_df['Max Temp']<100) & (reduced_df['Max Temp']>=75) & 
                        (reduced_df['Humidity']<=80) & (reduced_df['Humidity']>=40) &
                        (reduced_df['Wind Speed']<=10) & 
                        (reduced_df['Cloudiness']<=30) & (reduced_df['Cloudiness']>10)]    
reduced_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,hithadhoo,-0.6000,73.0833,82.80,69,25,8.01,MV,1619304876
150,batticaloa,7.7102,81.6924,78.85,80,22,2.84,LK,1619304897
246,uarini,-2.9900,-65.1083,84.20,74,20,4.61,BR,1619304924
348,singaraja,-8.1120,115.0882,78.62,80,11,4.52,ID,1619304947
363,namatanai,-3.6667,152.4333,83.46,71,30,7.94,PG,1619304925
438,bonthe,7.5264,-12.5050,81.01,78,13,8.46,SL,1619304969
447,mahajanga,-15.7167,46.3167,77.00,78,20,3.44,MG,1619304970


### Hotel Map

In [14]:
#create hotel df
hotel_df = reduced_df[['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ""

In [15]:
# find the closest hotel within 5000 meters of city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create for loop to go through each row
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "key": api_key
    }
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    print(f"Results for index {i}: {city}.")
    
    try:
        print(places_data["results"][0]['name'])
        hotel_df.loc[i, "Hotel Name"] = places_data["results"][0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[i, "Hotel Name"] = "NA"
        print("Results not found.")
        
    print("----------")
print("-------Done--------")

Results for index 58: hithadhoo.
Abuharee Grand
----------
Results for index 150: batticaloa.
Hotel East Lagoon
----------
Results for index 246: uarini.
Results not found.
----------
Results for index 348: singaraja.
Puri Bagus Lovina Resort
----------
Results for index 363: namatanai.
Namatanai Hotel
----------
Results for index 438: bonthe.
Bonthe Holiday Village
----------
Results for index 447: mahajanga.
Baobab Tree Hotel & Spa
----------
-------Done--------


In [17]:
#capitalize city names to make better-looking markers
hotel_df['City'] = [str.capitalize() for str in hotel_df['City']]
hotel_df

,City,Country,Lat,Lng,Hotel Name
58,Hithadhoo,MV,-0.6000,73.0833,Abuharee Grand
150,Batticaloa,LK,7.7102,81.6924,Hotel East Lagoon
246,Uarini,BR,-2.9900,-65.1083,NA
348,Singaraja,ID,-8.1120,115.0882,Puri Bagus Lovina Resort
363,Namatanai,PG,-3.6667,152.4333,Namatanai Hotel
438,Bonthe,SL,7.5264,-12.5050,Bonthe Holiday Village
447,Mahajanga,MG,-15.7167,46.3167,Baobab Tree Hotel & Spa


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))